In [7]:
import pandas as pd
import numpy as np
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score
import joblib
import os

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
label_encoder = joblib.load("label_encoder.pkl")

model_name = "deepset/gbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
num_labels = len(label_encoder.classes_)

def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)
train_ds = train_ds.map(preprocess_function, batched=True)
test_ds = test_ds.map(preprocess_function, batched=True)


model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)


training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True
)


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="macro")
    return {"accuracy": acc, "f1_macro": f1}


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

metrics = trainer.evaluate()
print(metrics)

save_dir = "gbert_email_classifier"
os.makedirs(save_dir, exist_ok=True)

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
joblib.dump(label_encoder, os.path.join(save_dir, "label_encoder.pkl"))

print(f"Trained model saved to: {save_dir}")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 500/500 [00:00<00:00, 10952.90 examples/s]
Some weights of the model checkpoint at deepset/gbert-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTr

{'loss': 3.5391, 'learning_rate': 1.4079336885731204e-05, 'epoch': 0.89}


 33%|███▎      | 563/1689 [36:14<55:22,  2.95s/it]  The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
                                                  
 33%|███▎      | 563/1689 [37:15<55:22,  2.95s/it]Saving model checkpoint to ./results/checkpoint-563
Configuration saved in ./results/checkpoint-563/config.json


{'eval_loss': 3.5100831985473633, 'eval_accuracy': 0.038, 'eval_f1_macro': 0.0021534625410857986, 'eval_runtime': 61.0321, 'eval_samples_per_second': 8.192, 'eval_steps_per_second': 1.032, 'epoch': 1.0}


Model weights saved in ./results/checkpoint-563/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-563/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-563/special_tokens_map.json
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
 59%|█████▉    | 1000/1689 [59:46<28:44,  2.50s/it] 

...

{'train_runtime': 5396.4909, 'train_samples_per_second': 2.502, 'train_steps_per_second': 0.313, 'train_loss': 3.524956874271481, 'epoch': 3.0}


100%|██████████| 63/63 [00:42<00:00,  1.47it/s]
Configuration saved in gbert_email_classifier/config.json


{'eval_loss': 3.5100831985473633, 'eval_accuracy': 0.038, 'eval_f1_macro': 0.0021534625410857986, 'eval_runtime': 43.4857, 'eval_samples_per_second': 11.498, 'eval_steps_per_second': 1.449, 'epoch': 3.0}


Model weights saved in gbert_email_classifier/pytorch_model.bin
tokenizer config file saved in gbert_email_classifier/tokenizer_config.json
Special tokens file saved in gbert_email_classifier/special_tokens_map.json


Trained model saved to: gbert_email_classifier


In [10]:
# TF-IDF + LogisticRegression baseline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

vec = TfidfVectorizer(max_features=20000, ngram_range=(1,2))
X_train = vec.fit_transform(train_df['text'])
X_test  = vec.transform(test_df['text'])
clf = LogisticRegression(max_iter=1000, class_weight='balanced')
clf.fit(X_train, train_df['label'])
pred = clf.predict(X_test)
print(classification_report(test_df['label'], pred))

              precision    recall  f1-score   support

           0       0.59      0.71      0.65        14
           1       0.62      0.62      0.62        13
           2       0.71      0.53      0.61        19
           3       0.79      0.73      0.76        15
           4       0.77      0.77      0.77        13
           5       0.88      0.88      0.88        17
           6       0.60      0.83      0.70        18
           7       0.76      0.84      0.80        19
           8       0.71      0.83      0.77        12
           9       0.75      0.75      0.75        16
          10       0.43      0.43      0.43        14
          11       0.92      0.73      0.81        15
          12       0.92      0.71      0.80        17
          13       0.56      0.71      0.62        14
          14       0.85      0.69      0.76        16
          15       0.81      0.68      0.74        19
          16       0.69      0.90      0.78        10
          17       0.71    

In [ ]:
import os
import random
import joblib
import numpy as np
import pandas as pd
import torch
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    set_seed
)
from datasets import Dataset

PREPARED_DIR = "prepared_data" 
MODEL_NAME = "deepset/gbert-base"
OUTPUT_DIR = "gbert_email_classifier_fixed"
MAX_LENGTH = 256
RANDOM_SEED = 42
NUM_EPOCHS = 4
BATCH_SIZE = 8
LEARNING_RATE = 2e-5

set_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

os.makedirs(OUTPUT_DIR, exist_ok=True)

train_df = pd.read_csv(os.path.join(PREPARED_DIR, "train.csv"))
test_df  = pd.read_csv(os.path.join(PREPARED_DIR, "test.csv"))
label_encoder = joblib.load(os.path.join(PREPARED_DIR, "label_encoder.pkl"))

print("Loaded train/test shapes:", train_df.shape, test_df.shape)
print("Classes in encoder:", len(label_encoder.classes_))

if train_df['label'].dtype != int and not np.issubdtype(train_df['label'].dtype, np.integer):
    train_df['label'] = train_df['label'].astype(int)
if test_df['label'].dtype != int and not np.issubdtype(test_df['label'].dtype, np.integer):
    test_df['label'] = test_df['label'].astype(int)

num_labels = len(label_encoder.classes_)
print("num_labels =", num_labels)
print("Train label distribution:\n", train_df['label'].value_counts().sort_index().head(30))

hf_train = Dataset.from_pandas(train_df.reset_index(drop=True))
hf_test  = Dataset.from_pandas(test_df.reset_index(drop=True))

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess_function(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=MAX_LENGTH)

hf_train = hf_train.map(preprocess_function, batched=True)
hf_test  = hf_test.map(preprocess_function, batched=True)

if "label" in hf_train.column_names:
    hf_train = hf_train.rename_column("label", "labels")
if "label" in hf_test.column_names:
    hf_test = hf_test.rename_column("label", "labels")

keep_cols = {"input_ids", "attention_mask", "labels"}
cols_to_remove = [c for c in hf_train.column_names if c not in keep_cols]
if cols_to_remove:
    hf_train = hf_train.remove_columns(cols_to_remove)
    hf_test  = hf_test.remove_columns([c for c in hf_test.column_names if c not in keep_cols])

print("Train columns after preprocess:", hf_train.column_names)
print("Test columns after preprocess:", hf_test.column_names)

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)
model.eval()
sample = hf_train.select(range(min(8, len(hf_train)))).with_format("pt")
batch = {k: sample[k] for k in ("input_ids", "attention_mask")}
with torch.no_grad():
    out = model(**batch)
print("Sanity logits shape:", out.logits.shape, " expected second dim:", num_labels)
if out.logits.shape[1] != num_labels:
    raise RuntimeError("logits second dim != num_labels; check num_labels and model init")

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE * 2,
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    logging_dir=os.path.join(OUTPUT_DIR, "logs"),
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    seed=RANDOM_SEED,
    warmup_ratio=0.06,
    save_total_limit=2,
    fp16=torch.cuda.is_available()
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="macro")
    return {"accuracy": acc, "f1_macro": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_train,
    eval_dataset=hf_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

pred_out = trainer.predict(hf_test)
preds = np.argmax(pred_out.predictions, axis=-1)
print("Final eval metrics (HF):", pred_out.metrics)
print("\nClassification report on TEST:")
print(classification_report(hf_test["labels"], preds, target_names=label_encoder.classes_, digits=4))

trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
joblib.dump(label_encoder, os.path.join(OUTPUT_DIR, "label_encoder.pkl"))
print("Model, tokenizer and label_encoder saved to", OUTPUT_DIR)

Loaded train/test shapes: (4500, 2) (500, 2)
Classes in encoder: 34
num_labels = 34
Train label distribution:
 label
0     130
1     121
2     171
3     136
4     115
5     154
6     159
7     171
8     111
9     143
10    127
11    138
12    150
13    127
14    140
15    169
16     86
17    124
18    159
19    130
20    130
21    133
22    137
23     64
24     96
25    115
26    133
27    170
28    114
29    172
Name: count, dtype: int64


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /Users/princess/.cache/huggingface/hub/models--deepset--gbert-base/snapshots/d50cb1df9660ff2de1af8a5362d322b3d5a1a28a/config.json
Model config BertConfig {
  "_name_or_path": "deepset/gbert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_em

Train columns after preprocess: ['labels', 'input_ids', 'attention_mask']
Test columns after preprocess: ['labels', 'input_ids', 'attention_mask']


Some weights of the model checkpoint at deepset/gbert-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at deepset/gbert-base and are newly

Sanity logits shape: torch.Size([8, 34])  expected second dim: 34


...

{'train_runtime': 5990.2027, 'train_samples_per_second': 3.005, 'train_steps_per_second': 0.376, 'train_loss': 1.222945096437715, 'epoch': 4.0}


100%|██████████| 32/32 [00:40<00:00,  1.26s/it]
Saving model checkpoint to gbert_email_classifier_fixed
Configuration saved in gbert_email_classifier_fixed/config.json


Final eval metrics (HF): {'test_loss': 0.3391866385936737, 'test_accuracy': 0.882, 'test_f1_macro': 0.8818058479706438, 'test_runtime': 41.7755, 'test_samples_per_second': 11.969, 'test_steps_per_second': 0.766}

Classification report on TEST:
                                          precision    recall  f1-score   support

                   1st Level Onlinetools     0.9333    1.0000    0.9655        14
                   Account clarification     1.0000    0.7692    0.8696        13
                          Bank statement     0.7143    0.7895    0.7500        19
                            Calculations     0.9286    0.8667    0.8966        15
                               Call-back     0.9286    1.0000    0.9630        13
                            Cancellation     0.9412    0.9412    0.9412        17
                    Change customer data     0.9474    1.0000    0.9730        18
          Change in payment transactions     0.9048    1.0000    0.9500        19
                 

Model weights saved in gbert_email_classifier_fixed/pytorch_model.bin
tokenizer config file saved in gbert_email_classifier_fixed/tokenizer_config.json
Special tokens file saved in gbert_email_classifier_fixed/special_tokens_map.json
tokenizer config file saved in gbert_email_classifier_fixed/tokenizer_config.json
Special tokens file saved in gbert_email_classifier_fixed/special_tokens_map.json


Model, tokenizer and label_encoder saved to gbert_email_classifier_fixed


In [ ]:
import os
import random
import joblib
import numpy as np
import pandas as pd
import torch
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    set_seed
)
from datasets import Dataset

PREPARED_DIR = "prepared_data" 
MODEL_NAME = "deepset/gelectra-base"
OUTPUT_DIR = "gelectra_email_classifier"
MAX_LENGTH = 256
RANDOM_SEED = 42
NUM_EPOCHS = 4
BATCH_SIZE = 8
LEARNING_RATE = 2e-5

set_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

os.makedirs(OUTPUT_DIR, exist_ok=True)

train_df = pd.read_csv(os.path.join(PREPARED_DIR, "train.csv"))
test_df  = pd.read_csv(os.path.join(PREPARED_DIR, "test.csv"))
label_encoder = joblib.load(os.path.join(PREPARED_DIR, "label_encoder.pkl"))

print("Loaded train/test shapes:", train_df.shape, test_df.shape)
print("Classes in encoder:", len(label_encoder.classes_))

if train_df['label'].dtype != int and not np.issubdtype(train_df['label'].dtype, np.integer):
    train_df['label'] = train_df['label'].astype(int)
if test_df['label'].dtype != int and not np.issubdtype(test_df['label'].dtype, np.integer):
    test_df['label'] = test_df['label'].astype(int)

num_labels = len(label_encoder.classes_)
print("num_labels =", num_labels)
print("Train label distribution:\n", train_df['label'].value_counts().sort_index().head(30))

hf_train = Dataset.from_pandas(train_df.reset_index(drop=True))
hf_test  = Dataset.from_pandas(test_df.reset_index(drop=True))

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess_function(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=MAX_LENGTH)

hf_train = hf_train.map(preprocess_function, batched=True)
hf_test  = hf_test.map(preprocess_function, batched=True)

if "label" in hf_train.column_names:
    hf_train = hf_train.rename_column("label", "labels")
if "label" in hf_test.column_names:
    hf_test = hf_test.rename_column("label", "labels")

keep_cols = {"input_ids", "attention_mask", "labels"}
cols_to_remove = [c for c in hf_train.column_names if c not in keep_cols]
if cols_to_remove:
    hf_train = hf_train.remove_columns(cols_to_remove)
    hf_test  = hf_test.remove_columns([c for c in hf_test.column_names if c not in keep_cols])

print("Train columns after preprocess:", hf_train.column_names)
print("Test columns after preprocess:", hf_test.column_names)

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)
model.eval()
from torch.utils.data import DataLoader

hf_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
loader = DataLoader(hf_train, batch_size=8)
batch = next(iter(loader))
with torch.no_grad():
    out = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])
print(out.logits.shape)
if out.logits.shape[1] != num_labels:
    raise RuntimeError("logits second dim != num_labels; check num_labels and model init")

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE * 2,
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    logging_dir=os.path.join(OUTPUT_DIR, "logs"),
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    seed=RANDOM_SEED,
    warmup_ratio=0.06,
    save_total_limit=2,
    fp16=torch.cuda.is_available()
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="macro")
    return {"accuracy": acc, "f1_macro": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_train,
    eval_dataset=hf_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

pred_out = trainer.predict(hf_test)
preds = np.argmax(pred_out.predictions, axis=-1)
print("Final eval metrics (HF):", pred_out.metrics)
print("\nClassification report on TEST:")
print(classification_report(hf_test["labels"], preds, target_names=label_encoder.classes_, digits=4))

trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
joblib.dump(label_encoder, os.path.join(OUTPUT_DIR, "label_encoder.pkl"))
print("Model, tokenizer and label_encoder saved to", OUTPUT_DIR)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded train/test shapes: (4500, 2) (500, 2)
Classes in encoder: 34
num_labels = 34
Train label distribution:
 label
0     130
1     121
2     171
3     136
4     115
5     154
6     159
7     171
8     111
9     143
10    127
11    138
12    150
13    127
14    140
15    169
16     86
17    124
18    159
19    130
20    130
21    133
22    137
23     64
24     96
25    115
26    133
27    170
28    114
29    172
Name: count, dtype: int64


Map: 100%|██████████| 500/500 [00:00<00:00, 12689.40 examples/s]


Train columns after preprocess: ['labels', 'input_ids', 'attention_mask']
Test columns after preprocess: ['labels', 'input_ids', 'attention_mask']


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at deepset/gelectra-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

{'eval_loss': 2.5524351596832275, 'eval_accuracy': 0.36, 'eval_f1_macro': 0.21359333803229644, 'eval_runtime': 21.1401, 'eval_samples_per_second': 23.652, 'eval_steps_per_second': 1.514, 'epoch': 4.0}


100%|██████████| 2252/2252 [59:59<00:00,  1.60s/it]
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


{'train_runtime': 3599.4051, 'train_samples_per_second': 5.001, 'train_steps_per_second': 0.626, 'train_loss': 3.0628380512894786, 'epoch': 4.0}


100%|██████████| 32/32 [00:20<00:00,  1.59it/s]
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_

Final eval metrics (HF): {'test_loss': 2.5524351596832275, 'test_accuracy': 0.36, 'test_f1_macro': 0.21359333803229644, 'test_runtime': 20.9172, 'test_samples_per_second': 23.904, 'test_steps_per_second': 1.53}

Classification report on TEST:
                                          precision    recall  f1-score   support

                   1st Level Onlinetools     1.0000    0.0714    0.1333        14
                   Account clarification     0.3750    0.2308    0.2857        13
                          Bank statement     0.2429    0.8947    0.3820        19
                            Calculations     0.0000    0.0000    0.0000        15
                               Call-back     0.0000    0.0000    0.0000        13
                            Cancellation     0.9286    0.7647    0.8387        17
                    Change customer data     0.6296    0.9444    0.7556        18
          Change in payment transactions     0.4444    0.8421    0.5818        19
                  